In [2]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

# Load the dataset
df = pd.read_csv("ecommerce_recommendations.csv")

# Aggregate duplicate ratings by averaging them
df_aggregated = df.groupby(["User_ID", "Product_ID"], as_index=False).Rating.mean()

# Create the user-item rating matrix
rating_matrix = df_aggregated.pivot(index="User_ID", columns="Product_ID", values="Rating").fillna(0)

# Convert to NumPy array
ratings_np = rating_matrix.values

# Perform Singular Value Decomposition (SVD)
U, sigma, Vt = svds(ratings_np, k=10)  # Using 10 latent features
sigma = np.diag(sigma)

# Reconstruct predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
predicted_df = pd.DataFrame(predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

# Function to recommend products with multiple attribute filtering
def recommend_products_filtered(user_id, num_recommendations=5, product_type=None, seller=None, location=None):
    """Recommend top-N products for a user with optional filtering by Product Type, Seller, and Location."""
    if user_id not in predicted_df.index:
        return f"User {user_id} not found in dataset."

    # Get the user's predicted ratings and sort them in descending order
    user_predictions = predicted_df.loc[user_id].sort_values(ascending=False)

    # Merge with product metadata (Product Type, Seller, Location)
    product_metadata = df[["Product_ID", "Product_Type", "Seller", "Location"]].drop_duplicates()
    recommendations = user_predictions.reset_index().merge(product_metadata, on="Product_ID")

    # Apply filters if specified
    if product_type:
        recommendations = recommendations[recommendations["Product_Type"] == product_type]
    if seller:
        recommendations = recommendations[recommendations["Seller"] == seller]
    if location:
        recommendations = recommendations[recommendations["Location"] == location]

    # Get the top-N recommendations
    top_recommendations = recommendations.head(num_recommendations)

    return top_recommendations

# Example Usage:
# Recommend 5 products for User 1 filtered by 'Electronics' category and 'California' location
print(recommend_products_filtered(1, 5, product_type="Electronics", location="California"))


     Product_ID         1 Product_Type          Seller    Location
260         110  0.055385  Electronics  Barnes & Noble  California
284         122  0.023072  Electronics          Amazon  California
309         131  0.011450  Electronics         Walmart  California
443         125 -0.062571  Electronics        Best Buy  California
490         164 -0.240126  Electronics  Barnes & Noble  California
